In [ ]:
import os, openai, json, llm_utils, re
import pandas as pd
from tqdm import tqdm
from azure.storage.blob import BlobServiceClient
from Arxiv_API import Arxiv_API
from dotenv import load_dotenv

load_dotenv()
rootdir=os.getenv("rootdir")
gemini_key=os.getenv("gemini_key")
connection_str=os.getenv("connection_str")

blob_service_client = BlobServiceClient.from_connection_string(conn_str=connection_str)



/Users/tengli/Python/chatGPT/LLMVenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Download the PDF from Azure storage account
with open("../SampleData/Data.pdf", "wb") as pdf_file:
        blob_data = blob_service_client.get_blob_client(container="samplepicture", blob=f"Data/Article_2306.00251v1.pdf").download_blob()
        pdf_file.write(blob_data.readall())


# Or download pdf from Arxiv and save as metadata
data=Arxiv_API(search=['math','physics','cs','econ','eess','q-bio'], start_date='2000-01-01', end_date='2024-12-31', start=0, sampling_unit_size=50, max_results=100, downloadstuff=True).df

In [13]:
from Arxiv_API import Arxiv_API
data=Arxiv_API(search=['physics'], start_date='2000-01-01', end_date='2024-12-31', start=0, sampling_unit_size=50, max_results=100, downloadstuff=True).df

2025-01-20 19:57:44,044 - INFO - Feed title of metadata 0: ArXiv Query: search_query=cat:physics*&amp;id_list=&amp;start=0&amp;max_results=100
2025-01-20 19:57:44,044 - INFO - Feed last update of metadata 0: 2025-01-20T00:00:00-05:00
2025-01-20 19:57:44,045 - INFO - totalResults for query 0: 269951
2025-01-20 19:57:44,045 - INFO - totalResults for query 0: 100
2025-01-20 19:57:44,045 - INFO - totalResults for query 0: 0
2025-01-20 19:57:44,045 - INFO - Generating e-print metadata for Article http://arxiv.org/abs/2305.09528v1
2025-01-20 19:57:44,045 - INFO - PDF from metadata list 0 article 0 already exists.
2025-01-20 19:57:44,064 - INFO - Generating e-print metadata for Article http://arxiv.org/abs/2108.02213v3
2025-01-20 19:57:44,064 - INFO - PDF from metadata list 0 article 1 already exists.
2025-01-20 19:57:44,084 - INFO - Generating e-print metadata for Article http://arxiv.org/abs/1006.1128v3
2025-01-20 19:57:44,084 - INFO - PDF from metadata list 0 article 2 already exists.
2025

In [2]:
import json, pandas as pd
# Or load the data from metadata.json
with open('../SampleData/metadata.json') as outfile:
    outputs=json.load(outfile)

metadata=pd.DataFrame(outputs)
display(metadata)

,ID,PublishDate,Title,Authors,Journal_Ref,Comment,Abstract,Content,Primary_Cat,Category,PDF_link,file_path
0,2305.09528v1,2022-10-02T11:45:44Z,German to Spanish translation of Einstein's wo...,"Enrique M. Padilla, Birgit L. Emberger, Manuel...",No Journal_Ref,No Comment,In 1926 Albert Einstein gave a clear explanati...,German to Spanish translation of Einstein's wo...,physics.hist-ph,No Category,http://arxiv.org/pdf/2305.09528v1,/Users/tengli/Python/chatGPT/SampleData/physic...
1,2108.02213v3,2021-08-04T18:00:02Z,How a fake Kepler portrait became iconic,"Steven N. Shore, Václav Pavlík","Physics Today 74, 9, 10 (2021)",A few minor things were corrected (typos and p...,For several decades a portrait of Johannes Kep...,How a fake Kepler portrait became iconic\nStev...,physics.hist-ph,No Category,http://arxiv.org/pdf/2108.02213v3,/Users/tengli/Python/chatGPT/SampleData/physic...
2,1006.1128v3,2010-06-06T18:19:23Z,Scientific Metaphors in the journalistic disco...,"Osame Kinouchi, Angélica A. Mandrá",No Journal_Ref,"In portuguese, 20 pages, 2 figures, new versio...",Scientific education and divulgation not only ...,Metáforas científicas no discurso jornalístico...,physics.hist-ph,No Category,http://arxiv.org/pdf/1006.1128v3,/Users/tengli/Python/chatGPT/SampleData/physic...
3,2501.01527v1,2025-01-02T20:35:24Z,Review Article: Integral Role of Physics in Ad...,İzzet Sakallı,EMUJPharmSci Volume: 7 Issue: 3 Pages: 122-132...,11 pages,Physics plays a fundamental role in advancing ...,Review Article : Integral Role of Physics in ...,physics.med-ph,No Category,http://arxiv.org/pdf/2501.01527v1,/Users/tengli/Python/chatGPT/SampleData/physic...
4,0707.1167v1,2007-07-09T01:31:01Z,Hollywood Blockbusters: Unlimited Fun but Limi...,"C. J. Efthimiou, R. A. Llewellyn",No Journal_Ref,includes 28 pictures; invited article for the ...,"In this article, we examine specific scenes fr...",arXiv:0707.1167v1 [physics.soc-ph] 9 Jul 200...,physics.soc-ph,No Category,http://arxiv.org/pdf/0707.1167v1,/Users/tengli/Python/chatGPT/SampleData/physic...
5,0802.0625v1,2008-02-05T02:42:25Z,Ab Initio Study of Different Acid Molecules In...,"Aleksey A. Zakharenko, S. Karthikyan, K. S. Kim",No Journal_Ref,"22 pages, 2 figs, 2 tables, 90 references",Using the Gaussian-03 for ab initio calculatio...,"Aleksey A. Zakharenko, S. Karthikyan, K.S. Kim...",physics.chem-ph,No Category,http://arxiv.org/pdf/0802.0625v1,/Users/tengli/Python/chatGPT/SampleData/physic...
6,2407.00234v1,2024-06-28T21:11:09Z,Medium-scale thermospheric gravity waves in th...,"Garima Malhotra, Timothy Fuller-Rowell, Tzu-We...",No Journal_Ref,No Comment,This paper presents a study of the global medi...,manuscript submitted to JGR: Atmospheres\nMedi...,physics.space-ph,No Category,http://arxiv.org/pdf/2407.00234v1,/Users/tengli/Python/chatGPT/SampleData/physic...
7,0907.5140v2,2009-07-29T14:58:55Z,Climate Engineering Responses to Climate Emerg...,"J. J. Blackstock, D. S. Battisti, K. Caldeira,...",No Journal_Ref,"66 pp., 5 figs., published by Novim, Santa Bar...",Despite efforts to stabilize CO_2 concentratio...,Novim Climate Engineering Responses to Cli...,physics.soc-ph,No Category,http://arxiv.org/pdf/0907.5140v2,/Users/tengli/Python/chatGPT/SampleData/physic...
8,1508.01805v2,2015-08-07T20:14:00Z,Revealing pre-earthquake signatures in atmosph...,"Dimitar Ouzounov, Sergey Pulinets, Dmitry Davi...",No Journal_Ref,"8 pages, 5 figures, presented at IWEP2, May, 2...",We analyze retrospectively/prospectively the t...,\t\r 1\t\r Revealing pre-earthquake signatur...,physics.geo-ph,No Category,http://arxiv.org/pdf/1508.01805v2,/Users/tengli/Python/chatGPT/SampleData/physic...
9,1004.0499v1,2010-04-04T11:29:17Z,Coherent radiation reaction effects in laser-v...,"P. W. Smorenburg, L. P. J. Kamp, G. A. Geloni,...","Laser and Particle Beams 28, 553-562 (2010)","31 pages, 4 figures",The effects of coherently enhanced radiation r...,arXiv:1004.0499v1 [physics.acc-ph] 4 Apr 201...,physics.acc-ph,No Category,http://arx

In [ ]:
def GetTitleAuthor_gpt():
    engine="gpt-3.5-turbo"
    system_prompt = "Please extract the title and author names:"
    system_prompt_tokens = llm_utils.count_tokens(text=system_prompt, engine=engine)

    def getEachRes(row):
        input_max_tokens = 100

        raw_text = row['Content']
        user_prompt = llm_utils.get_tokens_between_indices(
            text=raw_text, 
            engine=engine, 
            max_tokens=input_max_tokens)
        print(user_prompt)
        
        response=llm_utils.get_gpt_response(user_prompt=user_prompt, 
                                            system_prompt=system_prompt,
                                            engine=engine,
                                            max_completion=500)
        print("ChatGPT Response:", response)
        return response

    results = []
    for i in tqdm(range(len(metadata)), desc="Processing rows"):
        row = metadata.iloc[i]
        result = getEachRes(row)
        results.append(result)

    # After all rows are processed, write the accumulated results
    with open("all_results.json", 'w') as f:
        json.dump(results, f)


GetTitleAuthor_gpt()

In [ ]:
def GetTitleAuthor_gemini(metadata):
    user_prompt = "Please extract the title and author names: "

    def getEachRes(row):
        input_max_tokens = 1000
        raw_text = row['Content']
        response=llm_utils.get_gemini_response(user_prompt=user_prompt+raw_text[0:input_max_tokens], api_key=gemini_key)
        
        print("Gemini response:", response)
        return response

    results = []
    for i in tqdm(range(len(metadata)), desc="Processing rows"):
        row = metadata.iloc[i]
        result = getEachRes(row)
        results.append(result)

    # After all rows are processed, write the accumulated results
    with open(os.path.join(rootdir, "results", "all_results.json"), 'w') as f:
        json.dump(results, f)


GetTitleAuthor_gemini(metadata=metadata)

In [5]:
with open(os.path.join(rootdir, 'results', 'all_results.json')) as outfile:
    outputs=json.load(outfile)

outputs

['Title: On randomized confidence intervals for the binomial probability\n\nAuthor: Paul Kabaila\n',
 '**Title:** Confidence-Aware Learning for Camouflaged Object Detection\n\n**Authors:** Jiawei Liu, Jing Zhang, Nick Barnes\n',
 '**Title:** A Weighted Model Confidence Set: Applications to Local and Mixture Model Confidence Sets\n\n**Authors:** Amir.T. Payandeh Najafabadi, Ghobad Barmalzan, Shahla Aghaei\n',
 'Title: Confident AI\n\nAuthor: Jim Davis\n',
 'Title: Simultaneous confidence bands for the integrated hazard function\n\nAuthors: Anna Dudek, Maciej Gołcwin, Jacek Leśkow\n']

In [6]:
def parse_to_dict(string_list):
    titles=[]
    authors=[]
    for string in string_list:
        match_title = re.search(r"Title:\s*(.*?)\n", string)
        if match_title:
            titles.append(match_title.group(1).strip().strip("**"))
        
        match_author = re.search(r"(Author|Authors):\s*(.*?)\n", string)
        if match_author:
            authors.append(match_author.group(2).strip().strip("**"))

    result={"Title": titles, "Author":authors}
    return pd.DataFrame(result)

output_dict = parse_to_dict(string_list=outputs)
pd.set_option('display.max_colwidth', 100)
display(output_dict)

,Title,Author
0,On randomized confidence intervals for the binomial probability,Paul Kabaila
1,Confidence-Aware Learning for Camouflaged Object Detection,"Jiawei Liu, Jing Zhang, Nick Barnes"
2,A Weighted Model Confidence Set: Applications to Local and Mixture Model Confidence Sets,"Amir.T. Payandeh Najafabadi, Ghobad Barmalzan, Shahla Aghaei"
3,Confident AI,Jim Davis
4,Simultaneous confidence bands for the integrated hazard function,"Anna Dudek, Maciej Gołcwin, Jacek Leśkow"


In [5]:
def GetTitleAuthor_llama(metadata):
    user_prompt = "Extract the title and author names of an article with only clean answers like Title: xxx. the article is: "

    def getEachRes(row):
        input_max_tokens = 100
        raw_text = row['Content']
        response=llm_utils.get_llama_response(user_prompt=user_prompt+raw_text[0:input_max_tokens])
        return response

    results = []
    for i in tqdm(range(len(metadata)), desc="Processing rows"):
        row = metadata.iloc[i]
        result = getEachRes(row)
        print(result)
        results.append(result)

    # After all rows are processed, write the accumulated results
    with open(os.path.join(rootdir, "results", "all_results.json"), 'w') as f:
        json.dump(results, f)


GetTitleAuthor_llama(metadata=metadata.iloc[0:10,])

Processing rows:  10%|█         | 1/10 [00:01<00:17,  1.96s/it]

Here are the extracted information:

**Title:** German to Spanish translation of Einstein's work on the formation of meanders in rivers.

**Author:** Enrique M.


2025-01-21 00:52:07,683 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
Processing rows:  20%|██        | 2/10 [00:04<00:16,  2.03s/it]

Here are the extracted title and author names:

**Title:** How a fake Kepler portrait became iconic

**Authors:**

* Steven N. Shore
* Václav Pavlík


2025-01-21 00:52:09,528 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
Processing rows:  30%|███       | 3/10 [00:05<00:13,  1.94s/it]

...

Here are the extracted information:

**Title:** Climate Engineering Responses to Climate Emergencies
**Authors:**
1. Jason J. Blackstock
2. David S. Batt


2025-01-21 00:52:23,280 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
Processing rows:  90%|█████████ | 9/10 [00:19<00:02,  2.21s/it]

Here are the extracted information:

Title: Revealing pre-earthquake signatures in atmosphere and ionosphere associated with 2015 M7.8 

Author names not provided.


2025-01-21 00:52:27,113 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
Processing rows: 100%|██████████| 10/10 [00:23<00:00,  2.35s/it]

Here are the title and author names extracted from the article:

**Title:** Coherently enhanced radiation reaction effects in lasers

Unfortunately, there is no information on the authors provided in the snippet. If you need to know the authors' names, I can suggest checking the full text of the article or searching for it on arXiv.org.


In [7]:
metadata.iloc[0:2,]


,ID,PublishDate,Title,Authors,Journal_Ref,Comment,Abstract,Content,Primary_Cat,Category,PDF_link,file_path
0,2305.09528v1,2022-10-02T11:45:44Z,German to Spanish translation of Einstein's wo...,"Enrique M. Padilla, Birgit L. Emberger, Manuel...",No Journal_Ref,No Comment,In 1926 Albert Einstein gave a clear explanati...,German to Spanish translation of Einstein's wo...,physics.hist-ph,No Category,http://arxiv.org/pdf/2305.09528v1,/Users/tengli/Python/chatGPT/SampleData/physic...
1,2108.02213v3,2021-08-04T18:00:02Z,How a fake Kepler portrait became iconic,"Steven N. Shore, Václav Pavlík","Physics Today 74, 9, 10 (2021)",A few minor things were corrected (typos and p...,For several decades a portrait of Johannes Kep...,How a fake Kepler portrait became iconic\nStev...,physics.hist-ph,No Category,http://arxiv.org/pdf/2108.02213v3,/Users/tengli/Python/chatGPT/SampleData/physic...
